In [59]:
# This script demonstrates the capabilities of cluster_slug by using
# it to re-analyze the M31 star clusters catalog of Johnson et al. (2015, ApJ)

# Import the libraries we'll need
from astropy.io import ascii
from astropy import table
from astropy.timeseries import LombScargle
from astropy.table import unique, Table, hstack, Column
import sys
from numpy import inf
import numpy as np
import numpy.ma as ma
import time
from matplotlib.pyplot import figure as fig
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from slugpy.cluster_slug import cluster_slug
from slugpy import * 
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [49]:
#M33 photometry data with errors
m33phot_test=Table.read('/Users/user/Dropbox/m33_zoo_data/data_v2/apm33_6phot_test.fits')

#Property Tables with different number of clusters per filters used in detections
m33_propTbl_6filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Prop_Tables/M33_Prop6filt.fits')
m33_propTbl_5filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Prop_Tables/M33_Prop5filt663cls.fits')
m33_propTbl_4filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Prop_Tables/M33_Prop4filt1049cls.fits')
m33_propTbl_2filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Prop_Tables/M33_Prop2filt2129cls.fits')


In [44]:
m33_prop_all=hstack([m33_propTbl_6filt,m33_propTbl_5filt,
                     m33_propTbl_4filt,m33_propTbl_2filt])
m33_prop_all

ID_1,Chi_2(6),Mass_PDF_P25(6),Mass_PDF_Median(6),Mass_PDF_P75(6),MassIQR(6),Age_PDF_P25(6),Age_PDF_Median(6),Age_PDF_P75(6),Age_IQR(6),AV_PDF_P25(6),AV_PDF_Median(6),AV_PDF_P75(6),AV_IQR(6),ID_2,Chi_2(5),Mass_PDF_P25(5),Mass_PDF_Median(5),Mass_PDF_P75(5),MassIQR(5),Age_PDF_P25(5),Age_PDF_Median(5),Age_PDF_P75(5),Age_IQR(5),AV_PDF_P25(5),AV_PDF_Median(5),AV_PDF_P75(5),AV_IQR(5),ID_3,Chi_2(4),Mass_PDF_P25(4),Mass_PDF_Median(4),Mass_PDF_P75(4),MassIQR(4),Age_PDF_P25(4),Age_PDF_Median(4),Age_PDF_P75(4),Age_IQR(4),AV_PDF_P25(4),AV_PDF_Median(4),AV_PDF_P75(4),AV_IQR(4),ID_4,Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),Age_IQR(2),AV_PDF_P25(2),AV_PDF_Median(2),AV_PDF_P75(2),AV_IQR(2)
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
8,0.9509673255610723,2.755878199916389,4.125891340079773,4.669172412903183,1.9132942129867945,6.661417500071932,7.480314972179788,8.236220331048578,1.5748028309766466,0.26388315860375433,0.6098587952263942,1.6477857050943139,1.3839025464905594,4,16.19306630461277,2.259838959512405,3.3700220213689405,4.692793329112897,2.432954369600492,6.283464820637536,7.858267651614183,9.590550765688494,3.307085945050958,4.712141343751981,24.581027904080724,44.44991446440947,39.73777312065749,8,0.47145980336639637,2.5669108702386807,3.818819429353497,4.7164142453226106,2.14950337508393,6.535433273593799,6.9763780662672605,8.236220331048578,1.7007870574547788,0.26388315860375433,0.6592838861724857,1.6477857050943139,1.3839025464905594,1,5.5820165699238955e-05,2.165355294673551,2.5905317864483943,3.440884769998081,1.27552947532453,6.094488480920338,6.377952990496135,7.165354405984458,1.07086592506412,0.41215843144202857,0.9558344318490339,1.548935523202131,1.1367770917601023
13,0.5771298825610106,3.110191943062092,3.936924010402065,4.362100502176908,1.251908559114816,7.07086623612586,7.669291311896986,8.299212444287644,1.228346208161784,0.31330824954984576,0.7581340680646683,1.4006602503638568,1.087352000814011,5,0.5784880072827326,2.307080791931832,3.724335764514643,4.220375004918627,1.913294212986795,6.629921443452399,7.952755821472782,8.645669067102506,2.0157476236501077,0.26388315860375433,0.6098587952263942,2.1420366145552276,1.8781534559514732,11,0.7127014359623628,2.1889762108832644,2.9212246133843838,3.511747518627222,1.3227713077439573,6.440945103735201,7.385826802321189,8.14173216118998,1.7007870574547788,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,2,2.831436498120221e-07,2.1417343784638376,2.5669108702386807,3.5589893510466486,1.417254972582811,6.094488480920338,6.440945103735201,8.015747934711847,1.9212594537915084,0.46158352238812,1.054684613741217,1.8454860688786794,1.3839025464905594
20,0.8190774092197395,2.307080791931832,3.157433775481519,3.6534730158855027,1.3463922239536705,6.692913556691464,7.795275538375117,8.36220455752671,1.669291000835245,0.21445806765766293,0.5110086133342113,1.4995104322560393,1.2850523645983765,7,0.14499692298950626,2.448806289190113,4.196754088708913,4.527446915644903,2.0786406264547903,7.102362292745393,8.519684840624375,8.99212568991737,1.8897633971719765,0.26388315860375433,0.6592838861724857,1.548935523202131,1.2850523645983767,13,0.028513445959586124,2.307080791931832,3.346401105159227,4.243995921128341,1.9369151291965085,6.377952990496135,7.102362292745393,8.14173216118998,1.7637791706938444,0.41215843144202857,1.104109704687308,2.043186432663045,1.6310280012210163,3,8.5776906548467e-06,2.118113462254124,2.5669108702386807,3.488126602417508,1.3700131401633842,6.157480594159405,6.692913

In [45]:
m33_prop_all.rename_column('ID_1', 'ID(6)')
m33_prop_all.rename_column('ID_2', 'ID(5)')
m33_prop_all.rename_column('ID_3', 'ID(4)')
m33_prop_all.rename_column('ID_4', 'ID(2)')
m33_prop_all

ID(6),Chi_2(6),Mass_PDF_P25(6),Mass_PDF_Median(6),Mass_PDF_P75(6),MassIQR(6),Age_PDF_P25(6),Age_PDF_Median(6),Age_PDF_P75(6),Age_IQR(6),AV_PDF_P25(6),AV_PDF_Median(6),AV_PDF_P75(6),AV_IQR(6),ID(5),Chi_2(5),Mass_PDF_P25(5),Mass_PDF_Median(5),Mass_PDF_P75(5),MassIQR(5),Age_PDF_P25(5),Age_PDF_Median(5),Age_PDF_P75(5),Age_IQR(5),AV_PDF_P25(5),AV_PDF_Median(5),AV_PDF_P75(5),AV_IQR(5),ID(4),Chi_2(4),Mass_PDF_P25(4),Mass_PDF_Median(4),Mass_PDF_P75(4),MassIQR(4),Age_PDF_P25(4),Age_PDF_Median(4),Age_PDF_P75(4),Age_IQR(4),AV_PDF_P25(4),AV_PDF_Median(4),AV_PDF_P75(4),AV_IQR(4),ID(2),Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),Age_IQR(2),AV_PDF_P25(2),AV_PDF_Median(2),AV_PDF_P75(2),AV_IQR(2)
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
8,0.9509673255610723,2.755878199916389,4.125891340079773,4.669172412903183,1.9132942129867945,6.661417500071932,7.480314972179788,8.236220331048578,1.5748028309766466,0.26388315860375433,0.6098587952263942,1.6477857050943139,1.3839025464905594,4,16.19306630461277,2.259838959512405,3.3700220213689405,4.692793329112897,2.432954369600492,6.283464820637536,7.858267651614183,9.590550765688494,3.307085945050958,4.712141343751981,24.581027904080724,44.44991446440947,39.73777312065749,8,0.47145980336639637,2.5669108702386807,3.818819429353497,4.7164142453226106,2.14950337508393,6.535433273593799,6.9763780662672605,8.236220331048578,1.7007870574547788,0.26388315860375433,0.6592838861724857,1.6477857050943139,1.3839025464905594,1,5.5820165699238955e-05,2.165355294673551,2.5905317864483943,3.440884769998081,1.27552947532453,6.094488480920338,6.377952990496135,7.165354405984458,1.07086592506412,0.41215843144202857,0.9558344318490339,1.548935523202131,1.1367770917601023
13,0.5771298825610106,3.110191943062092,3.936924010402065,4.362100502176908,1.251908559114816,7.07086623612586,7.669291311896986,8.299212444287644,1.228346208161784,0.31330824954984576,0.7581340680646683,1.4006602503638568,1.087352000814011,5,0.5784880072827326,2.307080791931832,3.724335764514643,4.220375004918627,1.913294212986795,6.629921443452399,7.952755821472782,8.645669067102506,2.0157476236501077,0.26388315860375433,0.6098587952263942,2.1420366145552276,1.8781534559514732,11,0.7127014359623628,2.1889762108832644,2.9212246133843838,3.511747518627222,1.3227713077439573,6.440945103735201,7.385826802321189,8.14173216118998,1.7007870574547788,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,2,2.831436498120221e-07,2.1417343784638376,2.5669108702386807,3.5589893510466486,1.417254972582811,6.094488480920338,6.440945103735201,8.015747934711847,1.9212594537915084,0.46158352238812,1.054684613741217,1.8454860688786794,1.3839025464905594
20,0.8190774092197395,2.307080791931832,3.157433775481519,3.6534730158855027,1.3463922239536705,6.692913556691464,7.795275538375117,8.36220455752671,1.669291000835245,0.21445806765766293,0.5110086133342113,1.4995104322560393,1.2850523645983765,7,0.14499692298950626,2.448806289190113,4.196754088708913,4.527446915644903,2.0786406264547903,7.102362292745393,8.519684840624375,8.99212568991737,1.8897633971719765,0.26388315860375433,0.6592838861724857,1.548935523202131,1.2850523645983767,13,0.028513445959586124,2.307080791931832,3.346401105159227,4.243995921128341,1.9369151291965085,6.377952990496135,7.102362292745393,8.14173216118998,1.7637791706938444,0.41215843144202857,1.104109704687308,2.043186432663045,1.6310280012210163,3,8.5776906548467e-06,2.118113462254124,2.5669108702386807,3.488126602417508,1.3700131401633842,6.157480594159405,6.69

In [46]:
m33_prop_all.write('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Prop_Tables/M33_Prop_All.fits')


In [50]:
m33_propTbl_2filt

ID,Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),Age_IQR(2),AV_PDF_P25(2),AV_PDF_Median(2),AV_PDF_P75(2),AV_IQR(2)
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,5.5820165699238955e-05,2.165355294673551,2.5905317864483943,3.440884769998081,1.27552947532453,6.094488480920338,6.377952990496135,7.165354405984458,1.07086592506412,0.41215843144202857,0.9558344318490339,1.548935523202131,1.1367770917601023
2,2.831436498120221e-07,2.1417343784638376,2.5669108702386807,3.5589893510466486,1.417254972582811,6.094488480920338,6.440945103735201,8.015747934711847,1.9212594537915084,0.46158352238812,1.054684613741217,1.8454860688786794,1.3839025464905594
3,8.5776906548467e-06,2.118113462254124,2.5669108702386807,3.488126602417508,1.3700131401633842,6.157480594159405,6.692913556691464,8.36220455752671,2.2047239633673046,0.3627333404959372,0.9558344318490339,1.796060977932588,1.4333276374366508
4,2.043548933000631e-05,2.259838959512405,3.181054691691232,4.621930580483756,2.362091620971351,6.157480594159405,7.543307085418854,9.43307048259083,3.2755898884314254,0.31330824954984576,1.0052595227951253,2.2903118873935018,1.977003637843656
5,3.7982451130408193e-06,2.1417343784638376,2.5669108702386807,3.488126602417508,1.3463922239536705,6.094488480920338,6.377952990496135,7.511811028799321,1.4173225478789826,0.41215843144202857,1.0052595227951253,1.6477857050943139,1.2356272736522853
6,2.091595919014693e-05,2.165355294673551,2.5905317864483943,3.488126602417508,1.3227713077439573,6.094488480920338,6.377952990496135,7.354330745701656,1.2598422647813177,0.41215843144202857,0.9558344318490339,1.5983606141482223,1.1862021827061937
7,2.669871212089093e-06,2.1889762108832644,2.732257283706675,4.196754088708913,2.007777877825649,6.094488480920338,6.440945103735201,8.519684840624375,2.425196359704037,0.41215843144202857,1.104109704687308,1.8949111598247705,1.482752728382742
8,0.0008304491800575871,2.354322624351259,3.2519174403203728,4.291237753547768,1.936915129196509,6.125984537539872,6.503937216974267,7.669291311896986,1.5433067743571138,0.26388315860375433,0.6592838861724857,1.202959886579491,0.9390767279757367
9,9.871403713734451e-05,2.165355294673551,2.6613945350775348,3.936924010402065,1.771568715728514,6.094488480920338,6.440945103735201,8.36220455752671,2.267716076606371,0.41215843144202857,1.104109704687308,1.9443362507708621,1.5321778193288336


In [56]:
M33_2filt=hstack([m33phot_test["ID"],m33_propTbl_2filt])

In [57]:
M33_2filt

ID_1,ID_2,Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),Age_IQR(2),AV_PDF_P25(2),AV_PDF_Median(2),AV_PDF_P75(2),AV_IQR(2)
int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,1,5.5820165699238955e-05,2.165355294673551,2.5905317864483943,3.440884769998081,1.27552947532453,6.094488480920338,6.377952990496135,7.165354405984458,1.07086592506412,0.41215843144202857,0.9558344318490339,1.548935523202131,1.1367770917601023
2,2,2.831436498120221e-07,2.1417343784638376,2.5669108702386807,3.5589893510466486,1.417254972582811,6.094488480920338,6.440945103735201,8.015747934711847,1.9212594537915084,0.46158352238812,1.054684613741217,1.8454860688786794,1.3839025464905594
3,3,8.5776906548467e-06,2.118113462254124,2.5669108702386807,3.488126602417508,1.3700131401633842,6.157480594159405,6.692913556691464,8.36220455752671,2.2047239633673046,0.3627333404959372,0.9558344318490339,1.796060977932588,1.4333276374366508
4,4,2.043548933000631e-05,2.259838959512405,3.181054691691232,4.621930580483756,2.362091620971351,6.157480594159405,7.543307085418854,9.43307048259083,3.2755898884314254,0.31330824954984576,1.0052595227951253,2.2903118873935018,1.977003637843656
5,5,3.7982451130408193e-06,2.1417343784638376,2.5669108702386807,3.488126602417508,1.3463922239536705,6.094488480920338,6.377952990496135,7.511811028799321,1.4173225478789826,0.41215843144202857,1.0052595227951253,1.6477857050943139,1.2356272736522853
6,6,2.091595919014693e-05,2.165355294673551,2.5905317864483943,3.488126602417508,1.3227713077439573,6.094488480920338,6.377952990496135,7.354330745701656,1.2598422647813177,0.41215843144202857,0.9558344318490339,1.5983606141482223,1.1862021827061937
7,7,2.669871212089093e-06,2.1889762108832644,2.732257283706675,4.196754088708913,2.007777877825649,6.094488480920338,6.440945103735201,8.519684840624375,2.425196359704037,0.41215843144202857,1.104109704687308,1.8949111598247705,1.482752728382742
8,8,0.0008304491800575871,2.354322624351259,3.2519174403203728,4.291237753547768,1.936915129196509,6.125984537539872,6.503937216974267,7.669291311896986,1.5433067743571138,0.26388315860375433,0.6592838861724857,1.202959886579491,0.9390767279757367
9,9,9.871403713734451e-05,2.165355294673551,2.6613945350775348,3.936924010402065,1.771568715728514,6.094488480920338,6.440945103735201,8.36220455752671,2.267716076606371,0.41215843144202857,1.104109704687308,1.9443362507708621,1.5321778193288336
